# ⚙️ Etapa 2: Pré-Processamento de Dados

# 📌 1. Tratamento de Valores Faltantes

#### 🎯 Objetivo

#### Preencher valores ausentes de forma segura e consistente, evitando erros durante o treinamento do modelo.

## 🔎 1.1 Identificação de faltantes

In [12]:
import pandas as pd

df = pd.read_csv("../data/datasets/students_performance.csv")

df.isnull().sum()

student_id              0
age                     0
gender                  0
parental_education      0
study_hours_week      293
attendance_rate       232
extracurricular         0
sleep_hours           266
previous_scores       127
tutoring                0
internet_quality      155
family_income         278
health_status           0
final_grade             0
dtype: int64

#### df.isnull() → cria uma matriz booleana onde True indica um valor faltante.

#### .sum() → soma os True, mostrando quantos valores estão faltando em cada coluna.

## 🧮 1.2 Separar colunas numéricas e categóricas

In [13]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

num_cols, cat_cols

(Index(['age', 'study_hours_week', 'attendance_rate', 'sleep_hours',
        'previous_scores', 'final_grade'],
       dtype='object'),
 Index(['student_id', 'gender', 'parental_education', 'extracurricular',
        'tutoring', 'internet_quality', 'family_income', 'health_status'],
       dtype='object'))

#### 🧠 Por que isso é importante?

#### numéricas → melhor usar mediana (robusta a outliers)

#### categóricas → melhor usar moda (categoria mais frequente)

## 🛠️ 1.3 Imputação Numérica (mediana)

In [14]:
from sklearn.impute import SimpleImputer

imputer_num = SimpleImputer(strategy='median')
df[num_cols] = imputer_num.fit_transform(df[num_cols])

#### 🧠 O que acontece aqui?

#### SimpleImputer(strategy='median') → calcula a mediana de cada coluna numérica.

#### .fit_transform() →

####   -- fit = aprende a mediana

####   -- transform = substitui NaN por essa mediana

#### Substitui de forma segura valores que poderiam quebrar o modelo.

## 🛠️ 1.4 Imputação Categórica (moda)

In [15]:
imputer_cat = SimpleImputer(strategy='most_frequent')
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

#### 🧠 Explicação:

#### Para texto, não existe média nem mediana.
#### A categoria mais comum faz sentido como preenchimento.
#### Evita criar valores inexistentes.

## 📌 2. Tratamento de Outliers

#### 🎯 Objetivo

#### Reduzir o impacto de valores extremos que possam distorcer os treinos do modelo.

#### Com base no dataset, colunas numéricas são:

#### -- age

#### -- study_hours_week

#### -- attendance_rate

#### -- previous_scores

#### -- sleep_hours

#### -- final_grade (não mexer! é a variável alvo)

## 🛠️ 2.1 Aplicar capping pelo método do IQR

In [16]:
import numpy as np

for col in num_cols:
    if col == 'final_grade':
        continue  # nunca alterar o target!!!

    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    df[col] = np.where(df[col] < lower, lower, df[col])
    df[col] = np.where(df[col] > upper, upper, df[col])

#### 🧠 O que esse código faz?

#### IQR = Q3 - Q1 → intervalo onde estão 50% dos dados.

#### Valores abaixo de Q1 - 1.5*IQR ou acima de Q3 + 1.5*IQR são considerados outliers.

#### np.where() → substitui valores fora dos limites pelos limites seguros.

#### Mantém distribuição consistente sem remover dados.

## 📌 3. Encoding das Variáveis Categóricas

#### Vamos mapear todas as categorias do dataset:

#### ✔️ Nominais (sem ordem):

#### -- gender

#### -- parental_education

#### -- tutoring

#### -- extracurricular

#### -- internet_quality

#### -- family_income

#### -- health_status

#### Todas elas recebem One-Hot Encoding.

## 🛠️ 3.1 Aplicar One-Hot Encoding

In [17]:
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)


#### 🧠 O que acontece aqui?

#### get_dummies() cria uma coluna para cada categoria (ex.: gender_M).

#### drop_first=True evita multicolinearidade retirando uma categoria de referência.

#### Converte texto → números, sem perder significado.

## 📌 4. Normalização das Variáveis Numéricas

#### 🎯 Objetivo

#### Colocar todas as features na mesma escala, essencial para modelos lineares e redes neurais.

## 🛠️ 4.1 Aplicar StandardScaler

In [18]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

joblib.dump(scaler, "../models/scaler.pkl")


['../models/scaler.pkl']

#### 🧠 O que esse código faz?

#### StandardScaler() transforma cada coluna para ter:

#### -- média = 0

#### -- desvio padrão = 1

#### Isso evita que features com valores altos dominem o modelo.

#### joblib.dump() salva o scaler → para aplicar depois em dados de teste e produção.

## 📌 5. Feature Engineering (Opcional, mas recomendado)

## 🔧 5.1 Criar “study_efficiency”

#### Combina tempo de estudo + frequência:

In [19]:
df['study_efficiency'] = df['study_hours_week'] * (df['attendance_rate'] / 100)


## 🔧 5.2 Criar “wellbeing_score”

#### Combina sono + saúde:

In [20]:
df['wellbeing_score'] = df['sleep_hours'] * df['health_status_Excellent'] \
                      + df['health_status_Good'] * 0.5


## 📌 6. Salvar dataset final

In [21]:
df.to_csv('../data/processed/students_clean.csv', index=False)

#### 🧠 O que faz?

#### Cria o dataset limpo, escalonado e pronto para modelagem.